In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d rishavdash/retail-demand-forecasting-dataset

Dataset URL: https://www.kaggle.com/datasets/rishavdash/retail-demand-forecasting-dataset
License(s): ODbL-1.0
  0% 0.00/1.76M [00:00<?, ?B/s]
100% 1.76M/1.76M [00:00<00:00, 35.2MB/s]


In [ ]:
from zipfile import ZipFile
dataset='/content/retail-demand-forecasting-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv("Retail_Dataset2.csv")

In [ ]:
df.head(5)

,Product_id,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Open,Promo,StateHoliday,SchoolHoliday,Petrol_price
0,786725,Product_0033,Whse_S,Category_005,01/03/2016,16000,1,0,0,0,91
1,786743,Product_1825,Whse_S,Category_006,01/03/2016,50000,1,0,0,0,85
2,786967,Product_0551,Whse_S,Category_030,01/03/2016,3000,1,0,0,0,85
3,786856,Product_0556,Whse_S,Category_030,01/03/2016,1000,1,0,0,0,93
4,899538,Product_1844,Whse_A,Category_018,01/03/2016,7,1,0,0,0,95


In [ ]:
# predicting Order_Demand for given product code based on
# StateHoliday, Open, Day_Type, SchoolHoliday

1.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

NameError: name 'pd' is not defined

In [ ]:
state_holiday_mapping = {'0': 0, 'a': 1, 'b': 2}  # Example mapping
df['StateHoliday'] = df['StateHoliday'].map(state_holiday_mapping)
df['Day_Type'] = df['Day_Type'].astype(int)

In [ ]:
df1 = df.groupby(['Product_Code', 'Month', 'Year'], as_index=False).agg({
    'Order_Demand': 'sum',
    'Open': 'first',
    'StateHoliday': 'first',
    'SchoolHoliday': 'first',
    'Day_Type': 'first',
    'Petrol_price': 'mean'
})

df1 = df1.reset_index(drop=True)


In [ ]:
# df["Date"]=pd.to_datetime(df["Date"]) #, format='%d/%m/%Y'

2.

In [ ]:
df2=df.copy()
df2['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')

In [ ]:
df2['Year'] = df['Date'].dt.year
df2['Month'] = df['Date'].dt.month
df2['Day'] = df['Date'].dt.day

In [ ]:
# df2.drop('Date', axis = 1)

In [ ]:
df2.head(5)

,Product_id,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Open,Promo,StateHoliday,SchoolHoliday,Petrol_price,Day_Type,Year,Month,Day
0,786725,Product_0033,Whse_S,Category_005,2016-01-03,16000,1,0,0,0,91,1,2016,1,3
1,786743,Product_1825,Whse_S,Category_006,2016-01-03,50000,1,0,0,0,85,1,2016,1,3
2,786967,Product_0551,Whse_S,Category_030,2016-01-03,3000,1,0,0,0,85,1,2016,1,3
3,786856,Product_0556,Whse_S,Category_030,2016-01-03,1000,1,0,0,0,93,1,2016,1,3
4,899538,Product_1844,Whse_A,Category_018,2016-01-03,7,1,0,0,0,95,1,2016,1,3


In [ ]:
# df1 = pd.get_dummies(df, columns=['Product_Code', 'Warehouse', 'Product_Category'], drop_first=True)

In [ ]:
# df1.head()

In [ ]:
# #1 weekend, 0 weekday
# df2=df.copy()
# df2['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in ['Product_Code', 'Warehouse', 'Product_Category']:
    le = LabelEncoder()
    df2[column] = le.fit_transform(df2[column])
    label_encoders[column] = le

# 3.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

state_holiday_mapping = {'0': 0, 'a': 1, 'b': 2}
df['StateHoliday'] = df['StateHoliday'].map(state_holiday_mapping)
df['Day_Type'] = df['Day_Type'].astype(int)

df1 = df.groupby(['Product_Code', 'Month', 'Year'], as_index=False).agg({
    'Order_Demand': 'sum',
    'Open': 'first',
    'StateHoliday': 'first',
    'SchoolHoliday': 'first',
    'Day_Type': 'first',
    'Petrol_price': 'mean'
})

df1 = df1.reset_index(drop=True)

use df2

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1['Petrol_price_normalized_minmax'] = scaler.fit_transform(df1[['Petrol_price']])

# Display the normalized data
# print(df1[['Product_Code', 'Month', 'Year', 'Petrol_price', 'Petrol_price_normalized_minmax']])


In [ ]:
df2.head()

,Product_id,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Open,Promo,StateHoliday,SchoolHoliday,Petrol_price,Day_Type,Year,Month,Day
0,786725,32,3,2,2016-01-03,16000,1,0,0,0,91,1,2016,1,3
1,786743,1772,3,3,2016-01-03,50000,1,0,0,0,85,1,2016,1,3
2,786967,543,3,26,2016-01-03,3000,1,0,0,0,85,1,2016,1,3
3,786856,548,3,26,2016-01-03,1000,1,0,0,0,93,1,2016,1,3
4,899538,1791,0,15,2016-01-03,7,1,0,0,0,95,1,2016,1,3


In [ ]:
unique_classes = {}
for column in ['Product_Code', 'Warehouse', 'Product_Category']:
    unique_classes[column] = df[column].nunique()

for column, count in unique_classes.items():
    print(f"{column}: {count} unique classes")

Product_Code: 2117 unique classes
Warehouse: 4 unique classes
Product_Category: 30 unique classes


In [ ]:
df2.head()